This notebook contains code that builds a GBM model on the Motif-Topology segment of the data.
Classification: binomial.
PCA reduction used to reduce motif variables.
Note: We decided to pursue only group 1 and group 2 for prediction since Binary classification gives better prediction accuracy than a multinomial classification.


## GBM model of Motif-Topology Segment
### Data formating


In [ ]:
path <- "C:/Users/Sonali/Box Sync/Rscripts"
setwd(path)

library(dplyr)
library(gbm)

# .. data work from Sonali
df2 <- read.delim(file = "mergedMotifTop_withNA.txt", header = T, sep = "\t")
df2[1:5, 55:61]
rownames(df2) <- df2$X
predictorX <- df2[,2:61] # identify the columns representing the variables and convert to matrix.
dim(predictorX)
predictorX[1:5,1:10]
class(predictorX)

# Linking the uniprot IDS with their respective group labels.
predictorX$uniprot_swissprot <- row.names(predictorX) # create a column with uniprot IDs based on the row names.
class(predictorX$uniprot_swissprot) # must be character

dataf <- read.delim(file= "all_uniprotIDs.txt", header = TRUE, sep= "\t") ## Data frame containing Uniprot Swiss ID

dataf$uniprot_swissprot <- as.character(dataf$uniprot_swissprot) # convert factor to character.
class(dataf$uniprot_swissprot)# must be "character"

target <- left_join (predictorX,dataf,by="uniprot_swissprot") # this function links the unique protein ids with their group labels

mytarget <- target[,c(1:61,63)]
mytargetvariable <- mytarget[,62] # the target variable

### In-sample prediction with raw features Group 2"Mutant Type" Only

In [ ]:
x = predictorX[1:60] #dataframe of features
y = mytargetvariable #dependent variable

# recreate binary tags
y.wtOnly <- ifelse(y=="group1", 1, 0)
y.mtOnly <- ifelse(y=="group2", 1, 0)
table(y.wtOnly,y)
table(y.mtOnly,y)

train5 <- data.frame(y.mtOnly, x)

# binary classification
set.seed(1344)
fit2.gbm <- gbm(y.mtOnly~., data=train5, distribution = "bernoulli",
                n.trees = 5000, 
                interaction.depth = 1,
                n.minobsinnode = 1,
                shrinkage = 0.001,
                cv.folds = 5
)

gbm.perf(fit2.gbm)
summary(fit2.gbm)

# in-sample scoring
pred2.gbm.train5 = predict(fit2.gbm, newdata=train5, gbm.perf(fit2.gbm,plot.it=F),type="response")
summary(pred2.gbm.train5)
table(y.mtOnly)/length(y.mtOnly)

# in-sample ROC
library(pROC)
roc(train5$y.mtOnly, pred2.gbm.train5, plot= TRUE, 
    legacy.axes = TRUE, grid=c(0.1, 0.2), print.auc = T, 
    main = "GBM | Binary Target=MT Only | In-Sample")


### In-sample prediction with raw features Group 1 "Wild Type" Only

In [ ]:
x = predictorX[1:60] #dataframe of features
y = mytargetvariable #dependent variable

# recreate binary tags
y.wtOnly <- ifelse(y=="group1", 1, 0)
y.mtOnly <- ifelse(y=="group2", 1, 0)
table(y.wtOnly,y)
table(y.mtOnly,y)

train6 <- data.frame(y.wtOnly, x)

# binary classification
set.seed(1344)
fit2.gbm <- gbm(y.wtOnly~., data=train6, distribution = "bernoulli",
                n.trees = 5000, 
                interaction.depth = 1,
                n.minobsinnode = 1,
                shrinkage = 0.001,
                cv.folds = 5
)

gbm.perf(fit2.gbm)
summary(fit2.gbm)

# in-sample scoring
pred2.gbm.train6 = predict(fit2.gbm, newdata=train6, gbm.perf(fit2.gbm,plot.it=F),type="response")
summary(pred2.gbm.train6)
table(y.wtOnly)/length(y.wtOnly)

# in-sample ROC
library(pROC)
roc(train6$y.wtOnly, pred2.gbm.train6, plot= TRUE, 
    legacy.axes = TRUE, grid=c(0.1, 0.2), print.auc = T, 
    main = "GBM | Binary Target=WT Only | In-Sample")


### PCA analysis on Motif variables

In [ ]:
# .. PCA analysis to reduce motif variables
pc.motif <- prcomp(train5[,12:61])
summary(pc.motif)
plot(pc.motif, type='l') # .. scree plot ==> keep top 7

# check R's PCA list object
names(pc.motif)
dim(train5[,12:61]) # raw motif data = [48, 50]

# variable loadings: projections of the original variable onto the PC-space
class(pc.motif$rotation)
dim(pc.motif$rotation) # [50, 48]: loading of each of 50 variables to 48 PCs
head(pc.motif$rotation)

# data used in PCA: original 35 records now in 35 PC-space
class(pc.motif$x)
dim(pc.motif$x) # [48, 48]: 48 records onto 48 PCs, instead of the original 50 variables
head(pc.motif$x)

# variance explained by top PCs
pc.motif$sdev/sum(pc.motif$sdev)*100
cumsum(pc.motif$sdev/sum(pc.motif$sdev)*100)
plot(pc.motif$sdev/sum(pc.motif$sdev)*100) # .. full scree plot

# keep only top 12 PCs
pcs <- pc.motif$x
pcs12 <- pcs[,1:12]
head(pcs12)

# .. motif PC7 features - association with the tag
for (i in 1:12) {
  print(colnames(pcs12)[i])
  sid <- split(pcs12[,i],y)
  print(sapply(sid, summary, na.rm=T))
  cat("\n")
}

pcs12 <- pcs[,1:12]

### In-sample prediction with Motif PCA and topology-Group 1 - wild type Only

In [ ]:
x = cbind(predictorX[1:10], pcs12)
y = mytargetvariable #dependent variable

# recreate binary tags
y.wtOnly <- ifelse(y=="group1", 1, 0)
y.mtOnly <- ifelse(y=="group2", 1, 0)
table(y.wtOnly,y)
table(y.mtOnly,y)

# GBM binary classification for Group 1 - wild type Only
train1 <- data.frame(y.wtOnly, x)

# binary classification
set.seed(1344)
fit1.gbm <- gbm(y.wtOnly~., data=train1, distribution = "bernoulli",
                n.trees = 5000, 
                interaction.depth = 1,
                n.minobsinnode = 1,
                shrinkage = 0.001,
                cv.folds = 10
)

gbm.perf(fit1.gbm)
summary(fit1.gbm)
class(summary(fit1.gbm))
as.character(summary(fit1.gbm)$var)
list1 <- as.data.frame(summary(fit1.gbm))
list <- as.data.frame(summary(fit1.gbm))
finallist <- do.call(rbind,list); finallist


# in-sample scoring
pred1.gbm.train1 = predict(fit1.gbm, newdata=train1, gbm.perf(fit1.gbm,plot.it=F),type="response")
summary(pred1.gbm.train1)
table(y.wtOnly)/length(y.wtOnly)

# in-sample ROC
library(pROC)
roc(train1$y.wtOnly, pred1.gbm.train1, plot= TRUE, 
    legacy.axes = TRUE, grid=c(0.1, 0.2), print.auc = T, 
    main = "GBM | Binary Target=WT Only with PCA | In-Sample")
```
###In-sample prediction with Motif PCA and topology-Group 2 - Mutant type Only
```{r In-sample prediction with Motif PCA and Topology - Group 2}
x = cbind(predictorX[1:10], pcs12)
y = mytargetvariable #dependent variable

# recreate binary tags
y.wtOnly <- ifelse(y=="group1", 1, 0)
y.mtOnly <- ifelse(y=="group2", 1, 0)
table(y.wtOnly,y)
table(y.mtOnly,y)

train2 <- data.frame(y.mtOnly, x)

# binary classification
set.seed(1344)
fit2.gbm <- gbm(y.mtOnly~., data=train2, distribution = "bernoulli",
                n.trees = 5000, 
                interaction.depth = 1,
                n.minobsinnode = 1,
                shrinkage = 0.001,
                cv.folds = 10
)

gbm.perf(fit2.gbm)
summary(fit2.gbm)

# in-sample scoring
pred2.gbm.train2 = predict(fit2.gbm, newdata=train2, gbm.perf(fit2.gbm,plot.it=F),type="response")
summary(pred2.gbm.train2)
table(y.mtOnly)/length(y.mtOnly)

# in-sample ROC
library(pROC)
roc(train2$y.mtOnly, pred2.gbm.train2, plot= TRUE, 
    legacy.axes = TRUE, grid=c(0.1, 0.2), print.auc = T, 
    main = "GBM | Binary Target=MT Only with PCA | In-Sample")

### 10-fold cross-validation
10-fold cross-validation gave large performance variations in the AUC due to k-fold random seeds
AUC = 0.812, 0.7722, 0.7454, ...

In [ ]:
library(cvTools) #run the above line if you don't have this library
library(gbm)

k <- 10 #the number of folds

x = cbind(predictorX[1:10], pcs12)
y <- y.mtOnly
dataset <- data.frame(y,x) # create training dataset

set.seed(1234)
#set.seed(2345)
#set.seed(3456)
folds <- cvFolds(NROW(x), K=k)

kfoldpred <- rep(0,nrow(x))

for(i in 1:k){
  
  train <- x[folds$subsets[folds$which != i], ] #Set the training set 
  train_response <- y[folds$subsets[folds$which != i]] # set the training set response
  
  table(train_response)
  
  validation <- x[folds$subsets[folds$which == i], ] #Set the validation set
  
  randomseed = 7842 + (i-1)*10
  set.seed(randomseed)
  
  new_gbm.fit <- gbm(train_response~., data=train, distribution = "bernoulli",
                     n.trees = 5000,
                     interaction.depth = 1,
                     n.minobsinnode = 1,
                     shrinkage = 0.001,
                     cv.folds = 10)
  #Get your new GBM model (just fit on the train data)
  
  
  #gbm.perf(new_gbm.fit)
  new_gbmpred <- predict(new_gbm.fit, newdata=validation, 
                         gbm.perf(new_gbm.fit,plot.it=F),
                         type="response") #Get the predicitons for the validation set (from the model just fit on the train data)
  
  print(gbm.perf(new_gbm.fit,plot.it=F))
  kfoldpred[folds$subsets[folds$which == i]] <- new_gbmpred #Put the hold out prediction in the data set for later use
} 

kfoldpred# predictions for all proteins using k-fold validation !

# k-fold validated ROC
library(pROC)
roc(y.mtOnly, kfoldpred, plot= TRUE, 
    legacy.axes = TRUE, grid=c(0.1, 0.2), print.auc = T, 
    main = "GBM | Binary Target=MT Only | 10-Fold CV")

We therefore moved on to perform a repeated 10-fold cross validation for both the groups but changed the random seed 100 times
### Repeated 10-fold cross-validation using motifPCA and topology - Group 1 wild type only
##### NOTE: this loop takes a long time to run !

In [ ]:
library (gbm)
library(cvTools)
library(ROCR)
library(pROC)
# create empty roc plot to plot roc curves 
plot.roc(0:1, 0:1, type = "n", legacy.axes = TRUE,  main = "GBM | Binary Target=WT Only with PCA | repeated 10-Fold CV")


k <- 10 #the number of folds

x = cbind(predictorX[1:10], pcs12)
y <- y.wtOnly
#dataset <- data.frame(y,x) # create training dataset

set.seed(1234)
#set.seed(2345)
#set.seed(3456)
folds <- cvFolds(NROW(x), K=k)

nsim <- 1 # number of repetitions
myauc <- rep(0, nsim)

mypreds <- data.frame(matrix(0, nrow(x),ncol = 100)) # create a dataframe to store results of all 100 nsim repetitions
row.names(mypreds) <- row.names(x) # row names for the dataframe
names(mypreds) <- paste("K", (1:100), sep = "") # column names

j <- 1

x$kfoldlpred <- rep(0,nrow(x)) # append a column to original dataframe to temporarily store results of each k-fold

# Start the clock!
ptm <- proc.time()

repeatcv <- function(){
  while (j <= nsim){
    for(i in 1:k){
      
      train <- x[folds$subsets[folds$which != i], -23] #Set the training set 
      train_response <- y[folds$subsets[folds$which != i]] # set the training set response
      
      #table(train_response)
      
      validation <- x[folds$subsets[folds$which == i], -23] #Set the validation set
      
      randomseed = 7842 + (i-1)*10 +j
      set.seed(randomseed)
      
      new_gbm.fit <- gbm(train_response~., data=train, distribution = "bernoulli",
                         n.trees = 5000,
                         interaction.depth = 1,
                         n.minobsinnode = 1,
                         shrinkage = 0.001,
                         cv.folds = 10)
      #Get your new GBM model (just fit on the train data)
      
      #gbm.perf(new_gbm.fit)
      new_gbmpred <- predict(new_gbm.fit, newdata=validation, 
                             gbm.perf(new_gbm.fit,plot.it=F),
                             type="response") #Get the predicitons for the validation set (from the model just fit on the train data)
      
      #print(gbm.perf(new_gbm.fit,plot.it=F))
      x[folds$subsets[folds$which == i],]$kfoldlpred <- new_gbmpred #Put the hold out prediction in the data set for later use
    } 
    mypreds[,j] <- x$kfoldlpred
    rocobj2 <- roc(y.wtOnly, as.numeric(mypreds[,j]))
    myauc[j] <- rocobj2$auc # assign auc value to the jth item of your numeric vector 'myauc'
    plot.roc(rocobj2,add = TRUE)
    gbmsummary <- as.data.frame(summary(new_gbm.fit))
    j <- j+1
  }
  predictions <- as.data.frame(mypreds[,1],row.names = row.names(mypreds))
  returnlist = list(gbmsummary, predictions,myauc,mean(myauc), sd(myauc))
  returnlist
}

repeatcv()

proc.time() - ptm

### Repeated 10-fold cross-validation using motifPCA and topology - Group 2 mutant type only
#### NOTE: this loop takes a long time to run!

In [ ]:
library (gbm)
library(cvTools)
library(ROCR)
library(pROC)
# create empty roc plot to plot roc curves 
plot.roc(0:1, 0:1, type = "n", legacy.axes = TRUE,  main = "GBM | Binary Target=MT Only with PCA | repeated 10-Fold CV")


k <- 10 #the number of folds

x = cbind(predictorX[1:10], pcs12)
y <- y.mtOnly
#dataset <- data.frame(y,x) # create training dataset

set.seed(1234)
#set.seed(2345)
#set.seed(3456)
folds <- cvFolds(NROW(x), K=k)

nsim <- 1 # number of repetitions
myauc <- rep(0, nsim)

mypreds <- data.frame(matrix(0, nrow(x),ncol = 100)) # create a dataframe to store results of all 100 nsim repetitions
row.names(mypreds) <- row.names(x) # row names for the dataframe
names(mypreds) <- paste("K", (1:100), sep = "") # column names

j <- 1

x$kfoldlpred <- rep(0,nrow(x)) # append a column to original dataframe to temporarily store results of each k-fold

# Start the clock!
ptm <- proc.time()

repeatcv <- function(){
  while (j <= nsim){
    for(i in 1:k){
  
  train <- x[folds$subsets[folds$which != i], -23] #Set the training set 
  train_response <- y[folds$subsets[folds$which != i]] # set the training set response
  
  #table(train_response)
  
  validation <- x[folds$subsets[folds$which == i], -23] #Set the validation set
  
  randomseed = 7842 + (i-1)*10 +j
  set.seed(randomseed)
  
  new_gbm.fit <- gbm(train_response~., data=train, distribution = "bernoulli",
                     n.trees = 5000,
                     interaction.depth = 1,
                     n.minobsinnode = 1,
                     shrinkage = 0.001,
                     cv.folds = 10)
  #Get your new GBM model (just fit on the train data)
  
  
  #gbm.perf(new_gbm.fit)
  new_gbmpred <- predict(new_gbm.fit, newdata=validation, 
                         gbm.perf(new_gbm.fit,plot.it=F),
                         type="response") #Get the predicitons for the validation set (from the model just fit on the train data)
  
  #print(gbm.perf(new_gbm.fit,plot.it=F))
  x[folds$subsets[folds$which == i],]$kfoldlpred <- new_gbmpred #Put the hold out prediction in the data set for later use
    } 
    mypreds[,j] <- x$kfoldlpred
    rocobj2 <- roc(y.mtOnly, as.numeric(mypreds[,j]))
    myauc[j] <- rocobj2$auc # assign auc value to the jth item of your numeric vector 'myauc'
    plot.roc(rocobj2,add = TRUE)
    gbmsummary <- as.data.frame(summary(new_gbm.fit))
    j <- j+1
  }
  predictions <- as.data.frame(mypreds[,1],row.names = row.names(mypreds))
  returnlist = list(gbmsummary,predictions,myauc,mean(myauc), sd(myauc))
  returnlist
}

repeatcv()

proc.time() - ptm

### PCA analysis of motif variables using prcomp 

In [ ]:
res.pca <- prcomp(train5[,12:61])
names(res.pca)
#the standard deviations of the principal components (the square roots of the eigenvalues)
head(res.pca$sdev) 
#rotation : the matrix of variable loadings (columns are eigenvectors)
head(unclass(res.pca$rotation)[, 1:4])
#Variances of the principal componenets
#The variance retained by each principal component 
summary(res.pca)
# Eigenvalues
eig <- (res.pca$sdev)^2
# Variances in percentage
variance <- eig*100/sum(eig)
# Cumulative variances
cumvar <- cumsum(variance)
eig.decathlon2.active <- data.frame(eig = eig, variance = variance,
                     cumvariance = cumvar)
head(eig.decathlon2.active)


#### Eigenvalues of each PC

In [ ]:
#eigenvalues measures the variability retained by each PC
#The importance of princpal components (PCs) can be visualized with a scree plot
barplot(eig.decathlon2.active[, 2], names.arg=1:nrow(eig.decathlon2.active), 
       main = "Variances",
       xlab = "Principal Components",
       ylab = "Percentage of variances",
       col ="steelblue")
# Add connected line segments to the plot
lines(x = 1:nrow(eig.decathlon2.active), 
      eig.decathlon2.active[, 2], 
      type="b", pch=19, col = "red")

library(factoextra) # load library
fviz_screeplot(res.pca, ncp=10)# same plot using factoextra library

fviz_screeplot(res.pca, ncp=10, choice="eigenvalue") # can visualize eigenvalues instead of variances on Y-axis using factorextra

#### Coordinates of variables on the principal components

In [ ]:
#The correlation between variables and principal components is used as coordinates
# Helper function : 
# Correlation between variables and principal components
var_cor_func <- function(var.loadings, comp.sdev){
  var.loadings*comp.sdev
  }
# Variable correlation/coordinates
loadings <- res.pca$rotation
sdev <- res.pca$sdev
var.coord <- var.cor <- t(apply(loadings, 1, var_cor_func, sdev))
head(var.coord[, 1:4])

# Plot the correlation circle
a <- seq(0, 2*pi, length = 100)
plot( cos(a), sin(a), type = 'l', col="gray",
      xlab = "PC1",  ylab = "PC2")
abline(h = 0, v = 0, lty = 2)
# Add active variables
arrows(0, 0, var.coord[, 1], var.coord[, 2], 
      length = 0.1, angle = 15, code = 2)
# Add labels
text(var.coord, labels=rownames(var.coord), cex = 1, adj=1)

#### Cos2 : quality of representation for variables on the factor map

In [ ]:
var.cos2 <- var.coord^2
head(var.cos2[, 1:4])

# use of factoextra package to illustrate quality of variables in color based on the value of their cos2
fviz_pca_var(res.pca, col.var="contrib")+
scale_color_gradient2(low="white", mid="blue", 
      high="red", midpoint=55) + theme_minimal()

#### Contributions of the variables to the principal components

In [ ]:
comp.cos2 <- apply(var.cos2, 2, sum)
contrib <- function(var.cos2, comp.cos2){var.cos2*100/comp.cos2}
var.contrib <- t(apply(var.cos2,1, contrib, comp.cos2))
head(var.contrib[, 1:4])

# contribution of variables on PC1 - in a decreasing order.
var.contrib_PC4 <- var.contrib[order(var.contrib[, "PC4"], decreasing = TRUE),]

# Highlight the most important (i.e, contributing) variables :
fviz_pca_var(res.pca, col.var="contrib") +
scale_color_gradient2(low="white", mid="blue", 
      high="red", midpoint=50) + theme_minimal()

### Group 1- wild type only - Contribution of variables and individuals on PCs

In [ ]:
# contribution of top 10 variables on PC 4
fviz_contrib(res.pca, choice = "var", axes = 4, top = 5,xtickslab.rt = 90)
# visualize contribution of top 10 individuals on PCs
fviz_contrib(res.pca, choice = "ind", axes = 4, top = 5,xtickslab.rt = 90)

# contribution of top 10 variables on PC 10
fviz_contrib(res.pca, choice = "var", axes = 10, top = 5,xtickslab.rt = 90)
# visualize contribution of top 10 individuals on PCs
fviz_contrib(res.pca, choice = "ind", axes = 10, top = 5,xtickslab.rt = 90)

# contribution of top 10 variables on PC 6
fviz_contrib(res.pca, choice = "var", axes = 6, top = 5,xtickslab.rt = 90)
# visualize contribution of top 10 individuals on PCs
fviz_contrib(res.pca, choice = "ind", axes = 6, top = 5,xtickslab.rt = 90)

### Group 2- mutant type only - Contribution of variables and individuals on PCs

In [ ]:
# contribution of top 10 variables on PC 5
fviz_contrib(res.pca, choice = "var", axes = 5, top = 5,xtickslab.rt = 90)
# visualize contribution of top 10 individuals on PCs
fviz_contrib(res.pca, choice = "ind", axes = 5, top = 5,xtickslab.rt = 90)

# contribution of top 10 variables on PC 11
fviz_contrib(res.pca, choice = "var", axes = 11, top = 5,xtickslab.rt = 90)
# visualize contribution of top 10 individuals on PCs
fviz_contrib(res.pca, choice = "ind", axes = 11, top = 5,xtickslab.rt = 90)